## 데이터 수집

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import seaborn as sns
import pandas as pd
import numpy as np 
import scipy.stats as stats
from sklearn import datasets 
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from copy import deepcopy

In [2]:
data_df = pd.read_csv('../../datasets/titanic_disaster_train.csv')
data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 데이터 분석

## 데이터 전처리 

### 결측치 처리

In [3]:
data_df.dropna(inplace=True)

### 이상치 처리

### 범주형 처리

### 스케일링


## 데이터 분할

In [4]:
# PassengerId 유니크해서 통계적 의미 없으니 제외?
numeric_df = data_df.select_dtypes(exclude=['object'])
features = numeric_df.drop(columns=['Survived', 'PassengerId'])
label = data_df['Survived']
features.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [ ]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  183 non-null    int64  
 1   Age     183 non-null    float64
 2   SibSp   183 non-null    int64  
 3   Parch   183 non-null    int64  
 4   Fare    183 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 8.6 KB


## 모델 학습 

### 비선형 모델
- SVM(Support Vector Machine) : 고차원 공간에서 데이터를 비선형으로 분류해 표현
- SVM 하이퍼파라미터 많다 - 요새는 많이 좋아져서 default도 괜찮다

- 선형, 비선형, tree 

In [6]:
from sklearn.svm import SVC

model_svc = SVC(probability=True)
model_svc.fit(X=features,y=label)

SVC(probability=True)

### 앙상블 모델
- 독립 모델을 여러개 사용 : 집단지성
- 대체로 평균 값

In [7]:
from sklearn.ensemble import RandomForestClassifier

model_randomforestclassifier = RandomForestClassifier()
model_randomforestclassifier.fit(X=features,y=label)

RandomForestClassifier()

## 모델 평가 

### ROC, AUC 평가 방식 사용
- predict probability 값 중 양성 클래스 확률 값을 사용

##### label 카테고리(class)가 2개 일 때 사용하는 방식

### SVM model 

In [8]:
from sklearn.metrics import roc_curve, auc

In [9]:
predict_score = model_svc.predict_proba(X=features)[:,1] # 주로 수가 많은 쪽 class를 양성으로 정함
predict_score[:3]

array([0.6957227 , 0.69234294, 0.64095282])

In [10]:
FPR, TPR, threshold = roc_curve(y_true=label, y_score=predict_score)
FPR.shape, TPR.shape, threshold.shape

((82,), (82,), (82,))

In [11]:
# AUC(Area Under Curve) 구하기 : 면적 값이 1에 가까우면 좋음. (1 이면 과적합 의심)
auc_score = auc(FPR, TPR)
auc_score

0.7062330623306233

##### label 카테고리(class)가 3개 이상 일 때 사용하는 방식
- OVR, OVO 등이 있음.
- OVR(One vs Rest) : 하나의 양성 클래스에 나머지 음성 클래스 이걸 모든 클래스에 반복하고 최종 높은것 선택

In [12]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()
label = iris_dataset['target']
features = iris_dataset['data']
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
model_svc = SVC(probability=True
                , decision_function_shape='ovr')
model_svc.fit(X=features,y=label)

SVC(probability=True)

In [14]:
predict_score = model_svc.predict_proba(X=features) # 주로 수가 많은 쪽 class를 양성으로 정함
predict_score[:3]

array([[0.97726191, 0.01477096, 0.00796713],
       [0.9688639 , 0.02216439, 0.00897171],
       [0.97840058, 0.01384865, 0.00775077]])

In [15]:
from sklearn.metrics import roc_auc_score
AUC_score = roc_auc_score(y_true=label, y_score=predict_score, multi_class='ovr')
AUC_score

0.9981999999999999

In [16]:
model_svc = RandomForestClassifier()
model_svc.fit(X=features,y=label)

RandomForestClassifier()

In [17]:
predict_score = model_svc.predict_proba(X=features) # 주로 수가 많은 쪽 class를 양성으로 정함
predict_score[:3]


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [18]:
AUC_score = roc_auc_score(y_true=label, y_score=predict_score, multi_class='ovr')
AUC_score

1.0

## 모델 배포